In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random
import pandas as pd

In [3]:
cid = "d2800bc87fd74e95916e6605f18ccb80"
secret = "9b46ee2f9f434f91989f2055e2762d66"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


In [15]:
# Function to retrieve new releases in the US
def fetch_new_releases(limit):
    new_releases = sp.new_releases(country='US', limit=limit)
    album_uris = [album['uri'] for album in new_releases['albums']['items']]
    return album_uris

# Function to retrieve tracks for an album
def retrieve_tracks(album_uri):
    tracks_response = sp.album_tracks(album_uri)
    tracks = tracks_response['items']
    album_name = sp.album(album_uri)['name']  # Fetch album name using album URI
    for track in tracks:
        track['album_name'] = album_name  # Add album name to each track
    print(tracks)
    return tracks

# Function to retrieve audio features for a track
def audio_features(track_uri):
    features = sp.audio_features(track_uri)
    if features:
        track_info = sp.track(track_uri)
        popularity = track_info['popularity']
        features[0]['popularity'] = popularity  # Add popularity to the features dictionary
        return features[0]
    else:
        return None

# Main function to fetch data for new releases
def fetch_new_release_data(limit):
    all_data = []
    album_uris = fetch_new_releases(limit)
    for album_uri in album_uris:
        tracks = retrieve_tracks(album_uri)
        for track in tracks:
            track_uri = track['uri']
            track_features = audio_features(track_uri)
            if track_features:
                track_id = track_features['id']
                track_name = track['name']
                album_name = track['album_name'] if 'album_name' in track else None
                artist_name = track['artists'][0]['name']
                genres = sp.artist(track['artists'][0]['id'])['genres']
                data = {
                    'Track ID': track_id,
                    'Track Name': track_name,
                    'Album': album_name,
                    'Artist': artist_name,
                    'Genres': genres,
                    'Track URI': track_uri,
                    'Audio Features': track_features
                }
                all_data.append(data)
    return all_data

# Fetch data for new releases (adjust the limit as needed)
new_release_data = fetch_new_release_data(limit=5)

# Create DataFrame from the collected data
dataframe = pd.DataFrame(new_release_data)

# Optionally, you can flatten the 'Audio Features' dictionary
audio_features_df = pd.DataFrame(dataframe['Audio Features'].tolist())
dataframe.drop(columns=['Audio Features'], inplace=True)
dataframe = pd.concat([dataframe, audio_features_df], axis=1)

# Now 'dataframe' contains all the collected data merged into one DataFrame


[{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6l3HvQ5sa6mXTsMTB19rO5'}, 'href': 'https://api.spotify.com/v1/artists/6l3HvQ5sa6mXTsMTB19rO5', 'id': '6l3HvQ5sa6mXTsMTB19rO5', 'name': 'J. Cole', 'type': 'artist', 'uri': 'spotify:artist:6l3HvQ5sa6mXTsMTB19rO5'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/1vaQ6v3pOFxAIrFoPrAcom'}, 'href': 'https://api.spotify.com/v1/artists/1vaQ6v3pOFxAIrFoPrAcom', 'id': '1vaQ6v3pOFxAIrFoPrAcom', 'name': 'Ari Lennox', 'type': 'artist', 'uri': 'spotify:artist:1vaQ6v3pOFxAIrFoPrAcom'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/3ZooCJzNMTLpmJaIRUEorI'}, 'href': 'https://api.spotify.com/v1/artists/3ZooCJzNMTLpmJaIRUEorI', 'id': '3ZooCJzNMTLpmJaIRUEorI', 'name': 'Young Dro', 'type': 'artist', 'uri': 'spotify:artist:3ZooCJzNMTLpmJaIRUEorI'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/13y7CgLHjMVRMDqxdx0Xdo'}, 'href': 'https://api.spotify.com/v1/artists/13y7CgLHjMVRMDqxdx0X

In [16]:
dataframe

,Track ID,Track Name,Album,Artist,Genres,Track URI,danceability,energy,key,loudness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,popularity
0,2w5BtuWXISpB6NM4xPoeiR,"Pricey (feat. Ari Lennox, Young Dro & Gucci Mane)",Might Delete Later,J. Cole,"[conscious hip hop, hip hop, north carolina hi...",spotify:track:2w5BtuWXISpB6NM4xPoeiR,0.657,0.718,2,-6.880,...,0.310,78.956,audio_features,2w5BtuWXISpB6NM4xPoeiR,spotify:track:2w5BtuWXISpB6NM4xPoeiR,https://api.spotify.com/v1/tracks/2w5BtuWXISpB...,https://api.spotify.com/v1/audio-analysis/2w5B...,295147,4,77
1,1yfKakY4rvI17lk20ekuRA,Crocodile Tearz,Might Delete Later,J. Cole,"[conscious hip hop, hip hop, north carolina hi...",spotify:track:1yfKakY4rvI17lk20ekuRA,0.793,0.492,9,-8.583,...,0.198,86.978,audio_features,1yfKakY4rvI17lk20ekuRA,spotify:track:1yfKakY4rvI17lk20ekuRA,https://api.spotify.com/v1/tracks/1yfKakY4rvI1...,https://api.spotify.com/v1/audio-analysis/1yfK...,229187,4,78
2,4J9WveqdoaEUQu1aeoSH9X,Ready ‘24 (feat. Cam’ron),Might Delete Later,J. Cole,"[conscious hip hop, hip hop, north carolina hi...",spotify:track:4J9WveqdoaEUQu1aeoSH9X,0.449,0.697,11,-5.342,...,0.242,83.699,audio_features,4J9WveqdoaEUQu1aeoSH9X,spotify:track:4J9WveqdoaEUQu1aeoSH9X,https://api.spotify.com/v1/tracks/4J9WveqdoaEU...,https://api.spotify.com/v1/audio-analysis/4J9W...,210133,4,75
3,2cH8wHImerFokgmhjyhCTU,Huntin’ Wabbitz,Might Delete Later,J. Cole,"[conscious hip hop, hip hop, north carolina hi...",spotify:track:2cH8wHImerFokgmhjyhCTU,0.807,0.490,1,-8.451,...,0.431,113.982,audio_features,2cH8wHImerFokgmhjyhCTU,spotify:track:2cH8wHImerFokgmhjyhCTU,https://api.spotify.com/v1/tracks/2cH8wHImerFo...,https://api.spotify.com/v1/audio-analysis/2cH8...,162027,4,76
4,5RMTsrJkrtumtiIZjy7dL6,H.Y.B. (feat. Bas & Central Cee),Might Delete Later,J. Cole,"[conscious hip hop, hip hop, north carolina hi...",spotify:track:5RMTsrJkrtumtiIZjy7dL6,0.775,0.491,1,-7.395,...,0.309,139.897,audio_features,5RMTsrJkrtumtiIZjy7dL6,spotify:track:5RMTsrJkrtumtiIZjy7dL6,https://api.spotify.com/v1/tracks/5RMTsrJkrtum...,https://api.spotify.com/v1/audio-analysis/5RMT...,234587,4,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,5T7Gzs5ppc5eaHLGgsMvbn,Outside,Bryson Tiller,Bryson Tiller,"[kentucky hip hop, r&b, rap]",spotify:track:5T7Gzs5ppc5eaHLGgsMvbn,0.754,0.522,6,-7.613,...,0.793,98.029,audio_features,5T7Gzs5ppc5eaHLGgsMvbn,spotify:track:5T7Gzs5ppc5eaHLGgsMvbn,https://api.spotify.com/v1/tracks/5T7Gzs5ppc5e...,https://api.spotify.com/v1/audio-analysis/5T7G...,188580,4,57
76,5GDjl9jB48VB7NdcM8P0Gu,Undertow,Bryson Tiller,Bryson Tiller,"[kentucky hip hop, r&b, rap]",spotify:track:5GDjl9jB48VB7NdcM8P0Gu,0.459,0.410,3,-6.601,...,0.456,127.379,audio_features,5GDjl9jB48VB7NdcM8P0Gu,spotify:track:5GDjl9jB48VB7NdcM8P0Gu,https://api.spotify.com/v1/tracks/5GDjl9jB48VB...,https://api.spotify.com/v1/audio-analysis/5GDj...,238445,4,56
77,7iOrxO0nc8QANh6dNlyoEk,F4U,Bryson Tiller,Bryson Tiller,"[kentucky hip hop, r&b, rap]",spotify:track:7iOrxO0nc8QANh6dNlyoEk,0.607,0.793,2,-2.850,...,0.337,158.055,audio_features,7iOrxO0nc8QANh6dNlyoEk,spotify:track:7iOrxO0nc8QANh6dNlyoEk,https://api.spotify.com/v1/tracks/7iOrxO0nc8QA...,https://api.spotify.com/v1/audio-analysis/7iOr...,122706,4,57
78,7BOsIXFUa09SzoocKq9SoT,Assume The Position,Bryson Tiller,Bryson Tiller,"[kentucky hip hop, r&b, rap]",spotify:track:7BOsIXFUa09SzoocKq9SoT,0.784,0.861,1,-3.871,...,0.772,123.075,audio_features,7BOsIXFUa09SzoocKq9SoT,spotify:track:7BOsIXFUa09SzoocKq9SoT,https://api.spotify.com/v1/tracks/7BOsIXFUa09S...,https://api.spotify.com/v1/audio-analysis/7BOs...,201996,4,58


In [67]:
dataframe.columns

Index(['Track ID', 'Track Name', 'Artist', 'Genres', 'Track URI',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'popularity'],
      dtype='object')

In [9]:
dataframe["Album"].value_counts()

Series([], Name: Album, dtype: int64)